In [2]:
import getpass
import os


os.environ["MISTRAL_API_KEY"] ="v3hj7pDnJ8PqxOejAAYM2imTPIKhueWL"

In [3]:
%pip install -qU langchain_mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 12.6 MB/s eta 0:00:00


In [4]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="I love programming. = J'aime la programmation.\n\nHere's a breakdown:\n- I love = J'aime\n- programming = la programmation", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 27, 'total_tokens': 66, 'completion_tokens': 39}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-aaf37ce5-07b3-4657-953b-5f63cfe41e8e-0', usage_metadata={'input_tokens': 27, 'output_tokens': 39, 'total_tokens': 66})

**Identifying stakeholders**

In [6]:
problem_statement = f"""NewYork Metro station wants to establish a TicketDistributor machine that issues tickets for passengers
travelling in metro rails. Travelers have options of selecting a ticket for a single trip, round trips or for
multiple trips. They can also issue a metro pass for regular passengers or a time card for a day, a week or
a month according to their requirements. The discounts on tickets will be provided to frequent travelling
passengers. The machine is also supposed to read the metro pass and time cards issued by the metro
counters or machine. The ticket rates differ based on whether the traveler is a child or an adult. The
machine is also required to recognize original as well as fake currency notes. The typical transaction consists of a user using the display interface to select the type and quantity of tickets and then choosing a
payment method of either cash, credit/debit card or smartcard. The ticket or tickets are printed and
dispensed to the user. Also the messaging facilities after every transaction are required on the registered
number. The system can also be operated comfortably by a touch-screen. A large number of heavy
components are to be used. We do not want our system to slow down, and also usability of the machine.
The TicketDistributor must be able to handle several exceptions, such as aborting the transaction for
incomplete transactions, insufficient amount given by the travelers to the machine, money return in case
of aborted transaction, change return after successful transaction, showing insufficient balance in the
card, updated information printed on the tickets e.g. departure time, date, time, price, valid from, valid
till, validity duration, ticket issued from and destination station. In case of exceptions, an error message is
to be displayed. We do not want user feedback after every development stage but after every two stages
to save time. The machine is required to work in a heavy load environment such that at the morning and
evening time in weekdays, and in weekends performance and efficiency would not get affected."""

co_analyst_context = """ You are a requirements analyst powered by AI along with a human analyst and you need to identify the stakeholders and end-users"""

prompt = """Assume you are a requirement analyst responsible for developing requirements for a MetroTicketDistributor
system.
{co_analyst_context}
Using the problem statement below, generate the key stakeholders and end-users.

Problem Statement:
{problem_statement}
 """

stakeholders = llm.invoke(prompt)
print(stakeholders.content)

To generate the key stakeholders and end-users for the MetroTicketDistributor system, let's first consider the problem statement. Since the problem statement is not provided, I'll create a hypothetical one to illustrate the process:

**Problem Statement:**
The MetroTicketDistributor system aims to streamline the process of purchasing and managing metro tickets for commuters. The system should allow users to buy tickets online, check their ticket balance, and receive notifications about their travel history and upcoming promotions. Additionally, the system should support administrative functions for managing ticket sales, monitoring system performance, and handling customer support.

### Key Stakeholders:
1. **Commuters:**
   - **Description:** Individuals who use the metro system for daily travel.
   - **Interests:** Easy ticket purchasing, balance checking, travel history, and notifications.

2. **Metro Authority:**
   - **Description:** The governing body responsible for managing the

**Give elicitation techniques**

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses elicitation techniques to get requirements from {stakeholders} and list those elicitation techniques and justify why and how the chosen elicitation techniques are helpful in gathering requirements.
            Do it in the following format :
            Elicitation technique name -->
            Why :
            How :
            """
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

elicitation_tech = chain.invoke(
    {
     "stakeholders" : stakeholders,
     "input" : problem_statement,
    }
)

print(elicitation_tech.content)

### Elicitation Technique 1: Interviews
**Why:**
Interviews are essential for gathering detailed and specific information from key stakeholders. They allow for in-depth discussions and can help uncover hidden requirements and preferences.

**How:**
- Conduct structured interviews with representatives from each stakeholder group, including commuters, Metro Authority officials, system administrators, customer support representatives, and marketing team members.
- Ask open-ended questions to understand their needs, expectations, and pain points.
- Document the responses and analyze them to identify common themes and unique requirements.

### Elicitation Technique 2: Surveys/Questionnaires
**Why:**
Surveys and questionnaires are useful for collecting quantitative data from a large number of stakeholders. They can help identify trends and preferences across different user groups.

**How:**
- Design a survey with questions related to ticket purchasing preferences, payment methods, usability,

**Generate user stories**

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses elicitation techniques {elicitation_tech} to get all possible functionalities for each user in {stakeholders} in the form of user stories (both front and back of card). Therefore, list all possible user stories with the appropriate format.
            The front of the card will have the following format :
            As a [user role], I want to [goal] so that I can [reason].
            The back of the card will contain Success and Failure scenario.
            Use {example} for format.
             """
        ),
        ("human", "{input}"),
    ]
)

example = """Front of the Card :
As a registered	user I want to log in so that I	can	access subscriber‐only content
Back of the	Card :
1. Success - valid user logged in and referred to home page
(a) 'Remember me' ticked-store cookie/ automatic login next
time.
(b) 'Remember me' not ticked - force login next time.
2.   Failure-display message
(a) "Email address in wrong format"
(b) "Unrecognized user name, please try again"
(c) "Incorrect password, please try again"
(d) "Service unavailable, please try again"
(e) Account has expired - refer to account renewal sales page.
"""

chain = prompt | llm

generated_user_stories = chain.invoke(
    {
     "elicitation_tech" : elicitation_tech,
     "stakeholders" : stakeholders,
     "example" : example,
     "input" : problem_statement,
    }
)

print(generated_user_stories.content)

Based on the provided problem statement, here are the user stories for the MetroTicketDistributor system, following the specified format:

### Commuters

**Front of the Card:**
1. **As a** commuter, **I want to** select the type of ticket (single trip, round trip, multiple trips, metro pass, time card) **so that I can** purchase the ticket that suits my travel needs.
2. **As a** commuter, **I want to** select the quantity of tickets **so that I can** purchase multiple tickets in one transaction.
3. **As a** commuter, **I want to** choose my payment method (cash, credit/debit card, smartcard) **so that I can** complete my purchase.
4. **As a** commuter, **I want to** receive my tickets **so that I can** use them for my metro travel.
5. **As a** commuter, **I want to** receive a message on my registered number after every transaction **so that I can** keep track of my purchases.
6. **As a** commuter, **I want to** use a touch-screen interface **so that I can** interact with the system ea

**Validate user stories**

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses the INVEST framework to validate the generated user stories {generated_user_stories} i.e., make sure that the user stories follow the INVEST framework.
            The INVEST framework rules are :
1. All the stories that have been broken down are mostly independent	, hence making sure that they can be parallely developed.
2. Not every minute detail has been included in the cards, encouraging developers to seek further clarification and ensuring negotiability.
3. Stories have been written in regular business language that is equally valuable to both stakeholders and developers.
4. It is assumed that all the user stories mentioned below can be converted into working pieces of code within the working period of 1 to 3 weeks and hence are estimable	and small.
5. It is also written in a way where the completion of the story and its working is quantifiable and can be tested, hence being testable.
Give the validated user stories with this format :
The front of the card will have the following format :
            As a [user role], I want to [goal] so that I can [reason].
            The back of the card will contain Success and Failure scenario.
            Refer to {example}.
             """
        ),
        ("human", "{input}"),
    ]
)

example = """Front of the Card :
As a registered	user I want to log in so that I	can	access subscriber‐only content
Back of the	Card :
1. Success - valid user logged in and referred to home page
(a) 'Remember me' ticked-store cookie/ automatic login next
time.
(b) 'Remember me' not ticked - force login next time.
2.   Failure-display message
(a) "Email address in wrong format"
(b) "Unrecognized user name, please try again"
(c) "Incorrect password, please try again"
(d) "Service unavailable, please try again"
(e) Account has expired - refer to account renewal sales page.
"""

chain = prompt | llm

validated_user_stories = chain.invoke(
    {
     "generated_user_stories" : generated_user_stories,
     "stakeholders" : stakeholders,
     "example" : example,
     "input" : problem_statement,
    }
)

print(validated_user_stories.content)

Based on the provided problem statement and the INVEST framework, here are the validated user stories for the MetroTicketDistributor system:

### Commuters

**Front of the Card:**
1. **As a** commuter, **I want to** select the type of ticket (single trip, round trip, multiple trips, metro pass, time card) **so that I can** purchase the ticket that suits my travel needs.
2. **As a** commuter, **I want to** select the quantity of tickets **so that I can** purchase multiple tickets in one transaction.
3. **As a** commuter, **I want to** choose my payment method (cash, credit/debit card, smartcard) **so that I can** complete my purchase.
4. **As a** commuter, **I want to** receive my tickets **so that I can** use them for my metro travel.
5. **As a** commuter, **I want to** receive a message on my registered number after every transaction **so that I can** keep track of my purchases.
6. **As a** commuter, **I want to** use a touch-screen interface **so that I can** interact with the system

**Prioritize user stories**

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a requirements coanalyst that uses the MoSCoW technique to prioritize the validated user stories {validated_user_stories} taking into consideration the non-functional aspects as well.
            Display the user stories along with their MoSCoW counterparts, specify how, and provide details.
             """
        ),
        ("human", "{input}"),
    ]
)



chain = prompt | llm

prioritized_user_stories = chain.invoke(
    {
     "validated_user_stories" : validated_user_stories,
     "input" : problem_statement,
    }
)

print(prioritized_user_stories.content)

To prioritize the user stories using the MoSCoW technique, we need to categorize each user story into one of the following categories:

- **Must Have (M)**: Critical requirements that must be implemented for the system to function.
- **Should Have (S)**: Important requirements that should be implemented if possible, but the system can still function without them.
- **Could Have (C)**: Nice-to-have requirements that can be implemented if time and resources allow.
- **Won't Have (W)**: Requirements that are not necessary for the current release but might be considered for future releases.

Here are the user stories along with their MoSCoW counterparts:

### Commuters

**1. As a commuter, I want to select the type of ticket (single trip, round trip, multiple trips, metro pass, time card) so that I can purchase the ticket that suits my travel needs.**
   - **MoSCoW**: Must Have (M)
   - **How**: This is a core functionality that allows commuters to choose the type of ticket they need.
   -

**Identify EPICs**

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Identify three different EPICs (or collection of user stories) from {prioritized_user_stories}, where conflicts between the requirements occur. If the conflicts can be resolved, state solutions.
             """
        ),
        ("human", "{input}"),
    ]
)



chain = prompt | llm

epics = chain.invoke(
    {
     "prioritized_user_stories" : prioritized_user_stories,
     "input" : problem_statement,
    }
)

print(epics.content)

Based on the provided requirements and the user stories categorized using the MoSCoW technique, we can identify three different EPICs (collections of user stories) for the TicketDistributor machine project. Each EPIC will focus on a specific aspect of the system:

### EPIC 1: Core Ticket Purchasing Functionality
**User Stories:**
1. As a commuter, I want to select the type of ticket (single trip, round trip, multiple trips, metro pass, time card) so that I can purchase the ticket that suits my travel needs. (Must Have)
2. As a commuter, I want to choose my payment method (cash, credit/debit card, smartcard) so that I can complete my ticket purchase. (Must Have)
3. As a commuter, I want the machine to recognize original and fake currency notes so that I can avoid fraudulent transactions. (Must Have)
4. As a commuter, I want to receive change after a successful cash transaction so that I get the correct amount back. (Must Have)
5. As a commuter, I want to see an error message for any exc